In [1]:
import numpy as np 
import pandas as pd 
import os, re, random, zipfile, pathlib, cv2, PIL, pickle
np.random.seed(2)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
import PIL.Image
from sklearn.model_selection import train_test_split
import itertools
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from tensorflow import keras
from keras import layers, optimizers, models, layers, callbacks, regularizers
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, MaxPooling2D, GlobalMaxPooling2D, Activation
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications import VGG16
from keras.utils import to_categorical, load_img, img_to_array
from keras.wrappers.scikit_learn import KerasClassifier
from keras import models, layers, callbacks, regularizers
from keras.losses import binary_crossentropy
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, zero_one_loss
tf.keras.backend.set_floatx("float32")
%load_ext tensorboard
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
X = pickle.load(open('/kaggle/input/pured-pickle/x_data.pickle','rb'))
y = pickle.load(open('/kaggle/input/pured-pickle/y_data.pickle','rb'))

In [3]:
# resizing between [0,1]
X = X / 255.0

In [4]:
# preparing the dimentionality of y 
size = X.shape[0]
y = tf.reshape(tf.one_hot(y, 1), shape = [size, 1])

In [5]:
acc_model = []
loss_model = []
# split = 5
#KFold(n_splits= 5, shuffle=True, random_state= 2)
epochs = 5

In [6]:
base= tf.keras.applications.MobileNet(include_top = False, weights = "imagenet" ,input_shape=(64,64,3))

17225924/17225924 [==============================] - 0s 0us/step


In [7]:
model = Sequential([base, GlobalMaxPooling2D(), Dense(512, activation='relu'), Dropout(0.5), Dense(1, activation = "sigmoid")])
model.layers[0].trainable = False

In [8]:
#model = tf.keras.models.load_model('/kaggle/input/models/model.h5')
number = 1
for train, test in KFold(n_splits= 5, shuffle=True, random_state= 2).split(X, y):
    X_train = X[train]
    X_test =  X[test]
    y_train = y[train]
    y_test =  y[test]
    

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(learning_rate=1e-3),
                  metrics=['accuracy']
                 )
    history = model.fit(X_train, y_train, epochs = epochs , batch_size = 32, verbose = 0)
    prediction =  model.predict(X_test)
    y_prediction_np = tf.round(prediction).numpy()
    y_test_np = y_test.numpy()
    acc_model.append(accuracy_score(y_test_np, y_prediction_np))
    accuracy = accuracy_score(y_test_np, y_prediction_np)
    loss_model.append(zero_one_loss(y_test_np, y_prediction_np))
    loss = zero_one_loss(y_test_np, y_prediction_np)
    print(f' fold {number} has the accuracy of {accuracy}: and loss of {loss}')
    number = number + 1
print(f' The average of accuracy for 5 fold cross validation is: {np.mean(acc_model)}')
print(f' The average of loss for 5 fold cross validation is: {np.mean(loss_model)}')

156/156 [==============================] - 1s 5ms/step
 fold 1 has the accuracy of 0.8559118236472946: and loss of 0.14408817635270543
156/156 [==============================] - 1s 5ms/step
 fold 2 has the accuracy of 0.9011826017237924: and loss of 0.09881739827620761
156/156 [==============================] - 1s 5ms/step
 fold 3 has the accuracy of 0.9607135698536781: and loss of 0.03928643014632194
156/156 [==============================] - 2s 6ms/step
 fold 4 has the accuracy of 0.9821607536580477: and loss of 0.01783924634195233
156/156 [==============================] - 1s 5ms/step
 fold 5 has the accuracy of 0.9923832431348968: and loss of 0.007616756865103214
 The average of accuracy for 5 fold cross validation is: 0.9384703984035419
 The average of loss for 5 fold cross validation is: 0.06152960159645811
